In [1]:
import sqlite3
import pandas as pd
from itertools import combinations
from scipy.stats import chi2_contingency

### Tạo kết nối

In [2]:
# Tạo kết nối tới SQLite
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

### C1

In [3]:
# Tạo bảng dữ liệu A, B
cur.execute('''
create table scores (
    id integer primary key,
    A real,
    B real
)
''')
data = [
    (1, 8, 9),
    (2, 7.5, 8.5),
    (3, 6, 7),
    (4, 7, 6)
]
cur.executemany('insert into scores (id, A, B) values (?, ?, ?)', data)

query = '''
select
    (count(*) * sum(A * B) - sum(A) * sum(B)) /
    (sqrt(count(*) * sum(A*A) - sum(A)*sum(A)) * sqrt(count(*) * sum(B*B) - sum(B)*sum(B))) AS r_ab
FROM scores
'''
r_ab = cur.execute(query).fetchone()[0]
print("Pearson r_AB =", round(r_ab, 4))


Pearson r_AB = 0.6911


### C2

In [4]:
# Tạo bảng test_scores
cur.execute('''
create table test_scores (
    Day text,
    A real,
    B real,
    C real
)
''')

# Chèn dữ liệu
test_data = [
    ('Day 1', 8, 9, 7),
    ('Day 2', 7.5, 8.5, 7),
    ('Day 3', 6, 7, 8),
    ('Day 4', 7, 6, 5)
]
cur.executemany('insert into test_scores values (?, ?, ?, ?)', test_data)


In [5]:
# Chuyển đổi dữ liệu
query_long_format = '''
    select Day, 'A' as Model, A as Score from test_scores
    union all
    select Day, 'B', B from test_scores
    union all
    select Day, 'C', C from test_scores
'''

df_long = pd.read_sql_query(query_long_format, conn)

# Hiển thị kết quả sau khi chuyển đổi
print("Dữ liệu ở dạng quan hệ (long format):\n")
df_long

Dữ liệu ở dạng quan hệ (long format):



,Day,Model,Score
0,Day 1,A,8.0
1,Day 2,A,7.5
2,Day 3,A,6.0
3,Day 4,A,7.0
4,Day 1,B,9.0
5,Day 2,B,8.5
6,Day 3,B,7.0
7,Day 4,B,6.0
8,Day 1,C,7.0
9,Day 2,C,7.0


In [6]:
# Tạo bảng tần suất chéo giữa 'Day' và 'Model'
contingency = pd.crosstab(df_long['Day'], df_long['Model'])

# Thực hiện kiểm định Chi-squared
chi2, p, df, expected = chi2_contingency(contingency)

# Hiển thị kết quả
print(f"Chi-squared: {chi2:.4f}")
print(f"p-value    : {p:.4f}")
print(f"Bậc tự do  : {df}")
print("==> Kết luận:", "Có sự khác biệt đáng kể giữa các nhóm." if p < 0.05 else "Không có sự khác biệt đáng kể giữa các nhóm.")


Chi-squared: 0.0000
p-value    : 1.0000
Bậc tự do  : 6
==> Kết luận: Không có sự khác biệt đáng kể giữa các nhóm.


### C3

In [7]:
cur.execute("create table flights(flight_id integer, departure_time integer)")
flights_data = [(1, 830), (2, 1445), (3, 120), (4, 2359)]
cur.executemany("insert into flights values (?, ?)", flights_data)

# Tạo cột formatted_time từ departure_time (hhmm)
cur.execute("""
    select 
        flight_id,
        departure_time,
        printf('%02d:%02d', departure_time / 100, departure_time % 100) as formatted_time
    from flights
""")
flights_result = cur.fetchall()
df_flights = pd.DataFrame(flights_result, columns=["flight_id", "departure_time", "formatted_time"])
df_flights

,flight_id,departure_time,formatted_time
0,1,830,08:30
1,2,1445,14:45
2,3,120,01:20
3,4,2359,23:59


### C4

In [8]:
cur.execute("create table prices(price real)")
price_data = [
    (115.23,), 
    (115.79,), 
    (80.87,), 
    (82.75,), 
    (85.88,), 
    (300.00,)
]
cur.executemany("insert into prices values (?)", price_data)

df_prices = pd.read_sql_query("select * from prices", conn)

# Tính MAD và tìm ngoại lệ
median_price = df_prices["price"].median()
mad = (df_prices["price"] - median_price).abs().median()
threshold = 1.5 * mad
outliers = df_prices[(df_prices["price"] - median_price).abs() > threshold]

df_prices

,price
0,115.23
1,115.79
2,80.87
3,82.75
4,85.88
5,300.00


In [9]:
print("Giá trị ngoại lai:\n") 
outliers

Giá trị ngoại lai:



,price
5,300.0


### C5

In [10]:
cur.execute("create table Patient(id integer, last_name text, weight real, height real)")
patient_data = [
    (1, "Nguyen", 60, 170),
    (2, "Nguyen", 62, 172),
    (3, "Tran", 75, 180),
    (4, "Le", 80, 178)
]
cur.executemany("insert into Patient values (?, ?, ?, ?)", patient_data)

df_patients = pd.read_sql_query("select * from Patient", conn)
df_patients

,id,last_name,weight,height
0,1,Nguyen,60.0,170.0
1,2,Nguyen,62.0,172.0
2,3,Tran,75.0,180.0
3,4,Le,80.0,178.0


In [11]:
# Khoảng sai số chấp nhận cho weight (ví dụ ±1kg)
WEIGHT_TOLERANCE = 3.0

pairs = []
for (i, p1), (j, p2) in combinations(df_patients.iterrows(), 2):
    # So sánh theo khoảng cách Boolean:
    same_last_name = p1['last_name'] == p2['last_name']
    similar_weight = abs(p1['weight'] - p2['weight']) <= WEIGHT_TOLERANCE

    if same_last_name and similar_weight:
        pairs.append((p1['id'], p2['id'], p1['last_name'], p1['weight'], p2['weight']))

df_pairs = pd.DataFrame(pairs, columns=["id_1", "id_2", "last_name", "weight_1", "weight_2"])
df_pairs

,id_1,id_2,last_name,weight_1,weight_2
0,1,2,Nguyen,60.0,62.0


### Đóng kết nối

In [12]:
conn.close()